# Background on fine-tuning LLMs



![](https://archive.is/0iIXL/5f30742c57ad532b4cda9f1b48790dbcc7d00a85.webp)

**Summary:**

1. **LLM Pretraining:**
   - Large Language Models (LLMs) are pretrained on extensive text corpora.
   - Llama 2 was pretrained on a dataset of 2 trillion tokens, compared to BERT's training on BookCorpus and Wikipedia.
   - Pretraining is resource-intensive and time-consuming.

2. **Auto-Regressive Prediction:**
   - Llama 2, an auto-regressive model, predicts the next token in a sequence.
   - Auto-regressive models lack usefulness in providing instructions, leading to the need for instruction tuning.

3. **Fine-Tuning Techniques:**
   - Instruction tuning uses two main fine-tuning techniques:
     a. Supervised Fine-Tuning (SFT): Trained on instruction-response datasets, minimizing differences between generated and actual responses.
     b. Reinforcement Learning from Human Feedback (RLHF): Trained to maximize rewards based on human evaluations.

4. **RLHF vs. SFT:**
   - RLHF captures complex human preferences but requires careful reward system design and consistent human feedback.
   - Direct Preference Optimization (DPO) might be a future alternative to RLHF.
   - SFT can be highly effective when the model hasn't encountered specific data during pretraining.

5. **Effective SFT Example:**
   - LIMA paper showed improved performance of LLaMA v1 model over GPT-3 by fine-tuning on a small high-quality dataset.
   - Data quality and model size (e.g., 65b parameters) are crucial for successful fine-tuning.

6. **Importance of Prompt Templates:**
   - Prompt templates structure inputs: system prompt, user prompt, additional inputs, and model answer.
   - Llama 2's template example: <s>[INST] <<SYS>> System prompt <</SYS>> User prompt [/INST] Model answer </s>
   - Different templates (e.g., Alpaca, Vicuna) have varying impacts.

7. **Reformatting for Llama 2:**
   - Converting instruction dataset to Llama 2's template is important.
   - The tutorial author already reformatted a dataset for this purpose.

8. **Base Llama 2 Model vs. Chat Version:**
   - Specific prompt templates not necessary for base Llama 2 model, unlike the chat version.

(Note: LLMs = Large Language Models, SFT = Supervised Fine-Tuning, RLHF = Reinforcement Learning from Human Feedback, DPO = Direct Preference Optimization)

**Fine-Tuning Llama 2 (7 billion parameters) with VRAM Limitations and QLoRA:**

In this section, the goal is to fine-tune a Llama 2 model with 7 billion parameters using a T4 GPU with 16 GB of VRAM. Given the VRAM limitations, traditional fine-tuning is not feasible, necessitating parameter-efficient fine-tuning (PEFT) techniques like LoRA or QLoRA. The chosen approach is QLoRA, which employs 4-bit precision to drastically reduce VRAM usage.

The following steps will be executed:

1. **Environment Setup:**
   - The task involves leveraging the Hugging Face ecosystem and several libraries: transformers, accelerate, peft, trl, and bitsandbytes.

2. **Installation and Library Loading:**
   - The first step is to install and load the required libraries, as provided by Younes Belkada's GitHub Gist.

(Note: T4 GPU has 16 GB VRAM, 7 billion parameters of Llama 2 in 4-bit precision consume around 14 GB in FP16, and PEFT techniques like QLoRA are employed for efficient fine-tuning.)

# Install packages

In [1]:
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 wandb

# Authenticate with wandb, gdrive

In [2]:
import wandb
#create account and input ur key here
wandb.login(key='')

wandb: Currently logged in as: bertha-tam (298bwanderchat). Use `wandb login --relogin` to force relogin


True

In [3]:
#from google.colab import drive

#drive.mount('/content/drive')

In [4]:
#%mkdir RAFT_results

In [5]:
#%cd /content/drive/Shareddrives/'DATA 298 Team 7'/298B/
#%ls

# Import packages

In [6]:
# Import necessary packages for the fine-tuning process
import os                          # Operating system functionalities
import torch                       # PyTorch library for deep learning
from datasets import load_dataset,DatasetDict  # Loading datasets for training
from transformers import (
    AutoModelForCausalLM,          # AutoModel for language modeling tasks
    AutoTokenizer,                # AutoTokenizer for tokenization
    BitsAndBytesConfig,           # Configuration for BitsAndBytes
    HfArgumentParser,             # Argument parser for Hugging Face models
    TrainingArguments,            # Training arguments for model training
    pipeline,                     # Creating pipelines for model inference
    logging,                      # Logging information during training
)
from peft import LoraConfig, PeftModel  # Packages for parameter-efficient fine-tuning (PEFT)
from trl import SFTTrainer         # SFTTrainer for supervised fine-tuning
from datetime import datetime

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Input huggingface token, no need to Add token as git credential? n

In [7]:
#!pip install -q datasets
#!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/opt/conda/bin/huggingface-cl

In [8]:
!huggingface-cli whoami

beraht


In [9]:
import huggingface_hub
huggingface_hub.login(token='')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


---
* **Section 1:** Parameters to tune
    * Load a llama-2-7b-chat-hf model and train it on the chriztopherton/Reddit_RAFT dataset
    * The dataset contains 400+ samples.
    * You can find more information about the dataset in this notebook.
    * Feel free to use a different dataset.
* **Section 2:** QLoRA parameters
    * QLoRA will use a rank of 64 with a scaling parameter of 16.
    * See this article for more information about LoRA parameters.
    * The Llama 2 model will be loaded directly in 4-bit precision using the NF4 type.
    * The model will be trained for one epoch.
* **Section 3:** Other parameters
    * To get more information about the other parameters, check the [TrainingArguments](https://archive.is/o/0iIXL/https://huggingface.co/docs/transformers/main_classes/trainer%23transformers.TrainingArguments), [PeftModel](https://archive.is/o/0iIXL/https://huggingface.co/docs/peft/package_reference/peft_model), and [SFTTrainer](https://archive.is/o/0iIXL/https://huggingface.co/docs/trl/main/en/sft_trainer) documentation.

# Initialize model and dataset by name

In [10]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-hf"

# The instruction dataset to use
# if you have ur own custom dataset, follow this guide to publish it https://huggingface.co/docs/datasets/en/upload_dataset
dataset_name = "chriztopherton/Reddit_RAFT"

# Fine-tuned model name
new_model = "WanderChat_RAFT_epoch10_step20"

# Define LLM model parameters

In [11]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [12]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [13]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./RAFT_results"

In [14]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


1. **Loading the Dataset:**
   The first step involves loading the preprocessed dataset. This dataset will be used for fine-tuning. Preprocessing might involve reformatting prompts, filtering out low-quality text, and combining multiple datasets if needed.

2. **Configuring BitsAndBytes for 4-bit Quantization:**
   The `BitsAndBytesConfig` is set up to enable 4-bit quantization. This configuration is crucial for reducing the memory usage during fine-tuning.

3. **Loading Llama 2 Model and Tokenizer in 4-bit Precision:**
   The Llama 2 model is loaded with 4-bit precision, which significantly reduces the memory footprint. The corresponding tokenizer is also loaded to preprocess the text data.

4. **Loading Configurations and Initializing SFTTrainer:**
   - The configurations needed for QLoRA, which is a parameter-efficient fine-tuning technique, are loaded.
   - Regular training parameters are set up.
   - The `SFTTrainer` is initialized with all the loaded configurations and parameters. This trainer will manage the supervised fine-tuning process.

5. **Start of Training:**
   After all the necessary components are loaded and configured, the training process begins. The `SFTTrainer` takes care of fine-tuning the Llama 2 model using the specified dataset, configurations, and parameters.
   
  These steps collectively set up the environment for fine-tuning a Llama 2 model with 7 billion parameters in 4-bit precision using the QLoRA technique, thus optimizing for VRAM limitations while maintaining model performance.

# Load dataset and train/val/test split

In [15]:
# Step 1 : Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [16]:
train_test_dataset = dataset.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [17]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
        num_rows: 380
    })
    test: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
        num_rows: 22
    })
    valid: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
        num_rows: 21
    })
})

In [18]:
def transform_data(example):
    ques = example['instruction']
    ans = example['cot_answer']
    return {
        "text": f"<s>[INST] {ques} [/INST] {ans} </s>"
    }

transformed_dataset = train_test_valid_dataset.map(transform_data)
transformed_dataset

Map: 100%|██████████| 21/21 [00:00<00:00, 4481.78 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 380
    })
    test: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 22
    })
    valid: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 21
    })
})

# Define training procedures

In [19]:
# Step 2 :Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [20]:
# Step 3 :Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [21]:
# Step 4 :Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]


In [22]:
# Step 5 :Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [23]:
# Step 6 :Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [24]:
project = "Experiment_with_FineTuned_Instructions_20steps_10epochs"
base_model_name = "llama2"
run_name = base_model_name + "-" + project

# Number of training epochs
num_train_epochs = 10

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

evaluation_strategy = "steps"

eval_steps = 20

# Save checkpoint every X updates steps
save_steps = 20

# Log every X updates steps
logging_steps = 20

In [25]:
# Step 7 :Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    evaluation_strategy = evaluation_strategy,
    eval_steps=eval_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    report_to="wandb",
    #report_to="tensorboard",
    run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
)

In [26]:
# Step 8 :Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    eval_dataset=transformed_dataset['valid'],
    peft_config=peft_config,
    ### insert context and cot_anwer in dataset_text_field for model 
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 21/21 [00:00<00:00, 2391.21 examples/s]


# Train

In [27]:
# Step 9 :Train model
trainer.train()

# Step 10 :Save trained model
trainer.model.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,2.441500,2.221857
40,2.182300,2.130859
60,2.146200,2.102586
80,2.135000,2.085721
100,2.100800,2.072948
120,2.062600,2.064678
140,2.059800,2.055201
160,2.016000,2.049404
180,2.027300,2.037547
200,1.988400,2.040360


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [28]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.20s/it]


In [31]:
import huggingface_hub
huggingface_hub.login(token='')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [32]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 1.51M/9.98G [00:00<11:02, 15.0MB/s]
pytorch_model-00002-of-00002.bin:   0%|          | 1.49M/3.50G [00:00<03:54, 14.9MB/s]

pytorch_model-00001-of-00002.bin:   0%|          | 4.29M/9.98G [00:00<07:22, 22.6MB/s]
pytorch_model-00002-of-00002.bin:   0%|          | 4.34M/3.50G [00:00<02:32, 22.9MB/s]

pytorch_model-00001-of-00002.bin:   0%|          | 7.63M/9.98G [00:00<06:02, 27.5MB/s]
pytorch_model-00002-of-00002.bin:   0%|          | 7.78M/3.50G [00:00<02:04, 28.1MB/s]

pytorch_model-00001-of-00002.bin:   0%|          | 11.4M/9.98G [00:00<05:15, 31.5MB/s]
pytorch_model-00002-of-00002.bin:   0%|          | 11.4M/3.50G [00:00<01:51, 31.4MB/s]

pytorch_model-00001-of-00002.bin:   0%|          | 15.0M/9.98G [00:00<05:01, 33.0MB/

CommitInfo(commit_url='https://huggingface.co/beraht/WanderChat_RAFT_epoch10_step20/commit/518bdf4003ea44acc19383a9f949638b6e86209a', commit_message='Upload tokenizer', commit_description='', oid='518bdf4003ea44acc19383a9f949638b6e86209a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Load model directly
#from transformers import AutoTokenizer, AutoModelForCausalLM

#bertha_tokenizer = AutoTokenizer.from_pretrained("beraht/llama-2-7b-miniguanaco")
#bertha_model = AutoModelForCausalLM.from_pretrained("beraht/llama-2-7b-miniguanaco")